## Requests

In [1]:
import pandas as pd
import random
import os, time
from datetime import datetime 

import requests
import json
from bs4 import BeautifulSoup

### 變數

In [2]:
#不完整
input_city_nm = input('請輸入縣市名稱：')
if input_city_nm=='基隆市':
    city_num = 2
    #section = 'all'
elif input_city_nm=='台北市':
    city_num = 1
elif input_city_nm=='新北市':
    city_num = 3
    #section = [37,38]
elif input_city_nm=='桃園市':
    city_num = 6
elif input_city_nm=='新竹市':
    city_num = 4
elif input_city_nm=='台中市':
    city_num = 8
elif input_city_nm=='台南市':
    city_num = 15
    #section = [206,207,208,209,210,211]
elif input_city_nm=='高雄市':
    city_num = 17
else:
    print('請確認輸入的縣市是否正確')

請輸入縣市名稱：高雄市


In [3]:
min_time = 2
max_time = 4

In [4]:
sort_params = None
filter_params = None
page_num = 1
result = []
result_detail = []

### 紀錄 Cookie 取得 X-CSRF-TOKEN

In [30]:
s = requests.Session()
url = 'https://rent.591.com.tw/'

headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
}
r = s.get(url, headers=headers)

soup = BeautifulSoup(r.text, 'html.parser')
token_item = soup.select_one('meta[name="csrf-token"]')
#headers = headers.copy()
headers['X-CSRF-TOKEN'] = token_item.get('content')
#print(headers)

### 搜尋房屋

In [31]:
url_search = 'https://rent.591.com.tw/home/search/rsList'
params = 'is_format_data=1&is_new_list=1&type=1'

if filter_params:
    # 加上篩選參數，要先轉換為 URL 參數字串格式
    params += ''.join([f'&{key}={value}' for key, value, in filter_params.items()])
else:
    params += r'&region={}&firstRow={}'.format(city_num, page_num*30)
    # 在 cookie 設定地區縣市，避免某些條件無法取得資料

# params
# 似操作 dict：可以添加cookie，不會清除原cookie
#s.cookies['urlJumpIp'] = str(city_num)
# requests.cookies.set() 函数：會清除原有cookie
s.cookies.set('urlJumpIp', filter_params.get('region', str(city_num)) if filter_params else str(city_num), domain='.591.com.tw')
#s.cookies.set('urlJumpIp', str(city_num), domain='.591.com.tw')

# 排序參數
if sort_params:
    params += ''.join([f'&{key}={value}' for key, value, in sort_params.items()])

### 取得房屋詳情

In [34]:
#get web requests
res = s.get(url_search, params=params, headers=headers)
json_data = res.json()
tot_num = int(json_data['records'].replace(',',''))     #總筆數
want_page = int(tot_num/30)+1
all_data = json_data['data']['data']
print('總筆數：{}'.format(tot_num))
print('總頁數：{}'.format(want_page))
print('第 {} 頁開始'.format(page_num))
print('='*30)

for i in range(0, 30):
    data_num = i+1
    print('第 {} 筆'.format(data_num))
    print('縣市：{}'.format(input_city_nm))
    section_nm = all_data[i]['section_name']
    print('行政區：{}'.format(section_nm))
    location = all_data[i]['location'].replace('-','')
    print('地址：{}'.format(location))

    role_nm = all_data[i]['role_name']
    print('角色：{}'.format(role_nm))
    contact = all_data[i]['contact']
    print('姓名：{}'.format(contact))

    community = all_data[i]['community']
    print('社區：{}'.format(community))
    kind_nm = all_data[i]['kind_name']
    print('類型：{}'.format(kind_nm))
    area = all_data[i]['area']
    print('坪數：{}'.format(area))
    floor_str = all_data[i]['floor_str']
    print('樓層：{}'.format(floor_str))
    price = all_data[i]['price'].replace(',','') #+all_data[i]['price_unit']
    print('金額：{}'.format(price))

    title = all_data[i]['title']
    print('主旨：{}'.format(title))
    post_id = all_data[i]['post_id']
    print('網址：{}'.format(all_data[i]['post_id']))
    result.append(
                   dict(data_num=data_num, input_city_nm=input_city_nm, section_nm=section_nm, location=location, 
                        role_nm=role_nm, contact=contact, 
                        community=community, kind_nm=kind_nm, area=area, floor_str=floor_str, price=price,
                        title=title, post_id=post_id))
 
    print('='*30)
time.sleep(random.randint(min_time, max_time) + abs(random.random()))

總筆數：5127
總頁數：171
第 1 筆
縣市：高雄市
行政區：新興區
地址：新興區六合二路8號
角色：代理人
姓名：吳先生
社區：
類型：整層住家
坪數：18
樓層：3F/4F
金額：16000
主旨：美麗島捷運優質兩房
網址：13303294
第 2 筆
縣市：高雄市
行政區：鼓山區
地址：鼓山區龍德路
角色：代理人
姓名：謝先生
社區：埃及
類型：獨立套房
坪數：9
樓層：10F/10F
金額：11000
主旨：✨✨農16全新裝潢電梯大樓✨✨
網址：13381130
第 3 筆
縣市：高雄市
行政區：左營區
地址：左營區翠華路601巷69號
角色：屋主
姓名：楊小姐
社區：
類型：分租套房
坪數：5
樓層：9F/14F
金額：6000
主旨：近果貿市場及大義國中環境單純採光佳
網址：13273326
第 4 筆
縣市：高雄市
行政區：鳳山區
地址：鳳山區五甲一路
角色：代理人
姓名：李小姐
社區：
類型：分租套房
坪數：5
樓層：2F/5F
金額：5800
主旨：小雅房4500
網址：13348223
第 5 筆
縣市：高雄市
行政區：前鎮區
地址：前鎮區和昌街123號
角色：屋主
姓名：林先生
社區：
類型：獨立套房
坪數：7
樓層：3F/3F
金額：6000
主旨：高雄前鎮大套房.有電梯.免管理
網址：13307233
第 6 筆
縣市：高雄市
行政區：楠梓區
地址：楠梓區藍昌路360巷
角色：屋主
姓名：李小姐
社區：四季大樓
類型：整層住家
坪數：15.3
樓層：14F/15F
金額：16000
主旨：楠梓近德民市場兩房一廳景觀高樓層房
網址：13272605
第 7 筆
縣市：高雄市
行政區：鼓山區
地址：鼓山區明誠三路
角色：仲介
姓名：傅先生
社區：法蝶
類型：整層住家
坪數：15
樓層：4F/9F
金額：25000
主旨：（屋主自租）農十六公園旁大主臥2房+車位
網址：13355851
第 8 筆
縣市：高雄市
行政區：苓雅區
地址：苓雅區建國一路72巷
角色：屋主
姓名：吳先生
社區：
類型：分租套房
坪數：5
樓層：3F/5F
金額：6500
主旨：屋主自租!武廟商圈陽台三樓套房-門禁管理
網址：13381609
第 9 筆
縣市：高雄市
行政區：三民區
地址：三民區澄照街3號
角色：代理人
姓名：潘店長
社區：
類型：分租套房
坪數：

In [10]:
node_df = pd.DataFrame(result)
node_df = node_df.rename(columns={'data_num':'序號', 'input_city_nm':'縣市', 'section_nm':'行政區', 'location':'地址', 'role_nm':'角色', 
                                  'contact':'姓名', 'community':'社區', 'kind_nm':'類型', 'area':'坪數', 'floor_str':'樓層',
                                  'price':'金額', 'title':'主旨', 'post_id':'網址'})
node_df.head()
#node_df.shape

,序號,縣市,行政區,地址,角色,姓名,社區,類型,坪數,樓層,金額,主旨,網址
0,1,高雄市,左營區,左營區翠華路601巷69號,屋主,楊小姐,,分租套房,5,9F/14F,6000,近果貿市場及大義國中環境單純採光佳,13273326
1,2,高雄市,鼓山區,鼓山區美術東八街,屋主,侯先生,棋琴十三重奏,整層住家,25.2,2F/15F,27000,美術館輕軌大露台兩房,13381115
2,3,高雄市,左營區,左營區嘉慶街,屋主,侯先生,富國大街,整層住家,30,6F/14F,23800,義享學區捷運兩房+大平面車位,13380943
3,4,高雄市,苓雅區,苓雅區建民路222巷,代理人,黃先生,,獨立套房,8,5F/7F,9000,五塊厝捷運站武廟商圈,13349193
4,5,高雄市,三民區,三民區建國三路411號,仲介,王小姐,郡都FOCUS,整層住家,30,8F/15F,22500,精美三民市場國小全聯家樂福高雄火車站,13385482


In [11]:
for post_id in node_df['網址']:
    s = requests.Session()
    url = f'https://rent.591.com.tw/home/{post_id}'

    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
    }

    r = s.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    # print(soup)
    token_item = soup.select_one('meta[name="csrf-token"]')

    headers['X-CSRF-TOKEN'] = token_item.get('content')
    headers['deviceid'] = s.cookies.get_dict()['T591_TOKEN']
    headers['device'] = 'pc'
    # headers['token'] = s.cookies.get_dict()['PHPSESSID']

    url = f'https://bff.591.com.tw/v1/house/rent/detail?id={post_id}'
    res = s.get(url, headers=headers)
    house_detail = res.json()['data']

    room_str = house_detail['info'][0]['value']
    print('格局：{}'.format(room_str))
    
    kind_nm = house_detail['info'][3]['value']
    print('型態：{}'.format(kind_nm))
    
    phone = house_detail['linkInfo']['mobile']
    print('電話：{}'.format(phone))
    
    now_time = datetime.now().strftime('%y/%m/%d %H:%M:%S')
    print('時間：{}'.format(now_time))
    
    result_detail.append(dict(post_id=post_id, room_str=room_str, kind_nm=kind_nm, phone=phone, now_time=now_time))
    print('='*30)
    
    time.sleep(random.randint(min_time, max_time) + abs(random.random()))

格局：分租套房
型態：電梯大樓
電話：0968-954-919
時間：22/10/13 17:22:21
格局：2房2廳1衛
型態：電梯大樓
電話：0986-851-077,1666743
時間：22/10/13 17:22:25
格局：2房2廳1衛
型態：電梯大樓
電話：0986-851-077,1666729
時間：22/10/13 17:22:30
格局：獨立套房
型態：透天厝
電話：0913-080-300
時間：22/10/13 17:22:36
格局：3房1廳1衛
型態：電梯大樓
電話：0908-188-374
時間：22/10/13 17:22:40
格局：2房2廳2衛
型態：電梯大樓
電話：0986-851-077,1667236
時間：22/10/13 17:22:43
格局：獨立套房
型態：透天厝
電話：0986-851-077,1658878
時間：22/10/13 17:22:48
格局：獨立套房
型態：電梯大樓
電話：0937-393-533
時間：22/10/13 17:22:51
格局：分租套房
型態：透天厝
電話：0932-783-156
時間：22/10/13 17:22:56
格局：獨立套房
型態：電梯大樓
電話：
時間：22/10/13 17:23:02
格局：開放式格局
型態：電梯大樓
電話：0977-752-278
時間：22/10/13 17:23:07
格局：獨立套房
型態：電梯大樓
電話：0961-096-390
時間：22/10/13 17:23:10
格局：獨立套房
型態：電梯大樓
電話：0970-225-534
時間：22/10/13 17:23:15
格局：3房2廳2衛
型態：透天厝
電話：0912-768-537
時間：22/10/13 17:23:21
格局：獨立套房
型態：透天厝
電話：0919-524-779
時間：22/10/13 17:23:25
格局：獨立套房
型態：透天厝
電話：0970-156-166
時間：22/10/13 17:23:28
格局：2房1廳1衛
型態：透天厝
電話：0978-266-357
時間：22/10/13 17:23:34
格局：2房1廳1衛
型態：透天厝
電話：0963-256-658
時間：22/10/13 17:23:39
格局：獨立套房
型態：電梯大樓
電話：

In [12]:
node_detail = pd.DataFrame(result_detail)
node_detail = node_detail.rename(columns={'post_id':'網址', 'room_str':'格局', 'kind_nm':'型態', 'phone':'電話', 'now_time':'時間'})
node_detail.head()
#node_detail.shape

,網址,格局,型態,電話,時間
0,13273326,分租套房,電梯大樓,0968-954-919,22/10/13 17:22:21
1,13381115,2房2廳1衛,電梯大樓,"0986-851-077,1666743",22/10/13 17:22:25
2,13380943,2房2廳1衛,電梯大樓,"0986-851-077,1666729",22/10/13 17:22:30
3,13349193,獨立套房,透天厝,0913-080-300,22/10/13 17:22:36
4,13385482,3房1廳1衛,電梯大樓,0908-188-374,22/10/13 17:22:40


In [17]:
final_df_temp = node_df.merge(node_detail, how='inner', on=['網址'])
#final_df.shape

In [18]:
final_df = final_df.append(final_df_temp , ignore_index=True)
final_df.shape

(90, 17)

### 資料輸出

#### CSV檔

In [19]:
path = 'D:\GitHub_彙整\GitHub_2022_爬蟲_591租屋'

In [20]:
final_df.to_csv(r'{}\591租屋_{}_完整.csv'.format(path, input_city_nm), index=False, sep='^')

#### Excel檔

In [21]:
Result_PATH = r'{}\591租屋_{}_完整.xlsx'.format(path, input_city_nm)

# writing to Excel
student_result  = pd.ExcelWriter( Result_PATH, engine='xlsxwriter')

# write students data to excel
final_df.to_excel(student_result, sheet_name=input_city_nm, index=False, encoding='utf8')

# save the students result excel
student_result.save()
print('Students data is successfully written into Excel File')

Students data is successfully written into Excel File


#### 依條件篩選資料

In [23]:
df1 = pd.read_excel(r'{}\系統台分區.xlsx'.format(path))

In [24]:
df2 = final_df.merge(df1, how='inner', on=['縣市','行政區'])
df2.head()

,序號,縣市,行政區,地址,角色,姓名,社區,類型,坪數,樓層,金額,主旨,網址,格局,型態,電話,時間,系統,系統台區
0,1,高雄市,左營區,左營區翠華路601巷69號,屋主,楊小姐,,分租套房,5,9F/14F,6000,近果貿市場及大義國中環境單純採光佳,13273326,分租套房,電梯大樓,0968-954-919,22/10/13 17:22:21,D,DD
1,3,高雄市,左營區,左營區嘉慶街,屋主,侯先生,富國大街,整層住家,30,6F/14F,23800,義享學區捷運兩房+大平面車位,13380943,2房2廳1衛,電梯大樓,"0986-851-077,1666729",22/10/13 17:22:30,D,DD
2,12,高雄市,左營區,左營區辛亥路262號,代理人,吳小姐,華科和軒,獨立套房,7,3F/5F,10000,全新系統傢俱乾濕分離衛浴獨立小陽台,13259768,獨立套房,電梯大樓,0961-096-390,22/10/13 17:23:10,D,DD
3,21,高雄市,左營區,左營區實踐路20巷,屋主,郭先生,,分租套房,5,2F/4F,5000,交通便利出入自由房間寛場,13246621,分租套房,透天厝,0932-899-947,22/10/13 17:23:51,D,DD
4,23,高雄市,左營區,左營區文康路16號,屋主,黃先生,V Park,整層住家,20,4F/15F,25000,巨蛋時尚2+1房平車高樓層美景陽台,13275471,2房1廳1衛,電梯大樓,0984-464-051,22/10/13 17:24:02,D,DD


In [29]:
df2.iloc[:, 1:15].drop_duplicates().to_excel(r'{}\591租屋_{}_需求.xlsx'.format(path, input_city_nm), index=False, encoding='utf8')

## 參考資料

* [591 租屋網 - 搜尋房屋與房屋詳情](https://blog.jiatool.com/posts/house591_spider/)
* [python requests库添加自定义cookie的方法](https://blog.csdn.net/weixin_39888943/article/details/111079997?utm_medium=distribute.pc_aggpage_search_result.none-task-blog-2~aggregatepage~first_rank_ecpm_v1~rank_v31_ecpm-5-111079997-null-null.pc_agg_new_rank&utm_term=python%20request%20%E5%A4%9A%E4%B8%AAcookie&spm=1000.2123.3001.4430)
* [3個實用的Pandas套件清理重複資料教學](https://www.learncodewithmike.com/2022/01/pandas-drop-duplicate-data.html)